## Using the classificaction methods with features extracted from FIR filtered signals 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as sig
from scipy.io import loadmat

In [17]:
# carregar sinais e separar data e target
eeg = loadmat('/home/renato/Dropbox/Mestrado/final/kolodziej_dataset/dataset_15_canais.mat')
data = eeg['data']
target = eeg['target'].reshape((data.shape[0])) # mudando a dimensão do shape do vetor target

# projetar os bancos de filtros FIR
def design_fir_butter(cutoff, f_sampl, order=101, band='pass'):
    '''Design a butterworth filter with specified order. If the order is not
    specified, the value is 101. The band parameter options is 'pass' and 'stop'
    '''
    f_sampl = float(f_sampl)
    f_nyq = f_sampl / 2                   # Nyquist frequency
    cutoff = np.array(cutoff) / f_nyq     # Normalizing the cutoff frequency vector
    if order % 2 == 0:                    # ensures that the order is odd
        order = order + 1

    if band == 'pass':                    # defines DC gain (see signal.firwin
        pass_zero = False                 # documentation for details)
    elif band == 'stop':
        pass_zero = True
    else:
        print('ERROR: argument', band, 'not known')
    coeff = sig.firwin(order, cutoff, pass_zero=pass_zero, window='hamming')

    freq_vec, imp_vec = sig.freqz(coeff, 1.)  # Frequency response of the filter

    return coeff, freq_vec, imp_vec

def apply_fir_filter(coeff, input_array, axis):
    '''Apply filter coefficients to an input array. It is mathematically
    equivalent to make a convolution sum between the two systems.
    '''
    return sig.lfilter(coeff, 1., input_array, axis=axis)

def generate_band_tolerance(freq, tol):
    '''generates an array with the right cutoff frequencies for a bandpass or
    band reject filter, based on the desired tolerancy.
    '''
    freq = np.array(freq)
    f1 = freq - tol
    f2 = freq + tol
    freq_tol = []
    for i in range(len(f1[0])):
        for j in range(len(f2[0])):
            freq_tol.append([f1[i][j], f2[i][j]])
    return np.array(freq_tol)
   # return np.transpose(np.array([freq-tol, freq+tol])).reshape(2*len(freq))


def generate_harmonics(input_freq, number_of_harmonics):
    '''Generate the desired harmonics based on the specified number
    '''
    harmonics = []
    for i in input_freq:
        for j in range(number_of_harmonics):
            harmonics.append(i*(j+1))
    harmonics = np.array(harmonics)
    return harmonics.reshape(len(input_freq), number_of_harmonics)

# aplicar os filtros aos sinais 
# criar os estimadores
# utilizar os sinais filtrados como entrada para os estimadores 

In [34]:
freq_array = generate_harmonics([5, 6, 7, 8], 4)
x = freq_array.reshape((16,))
x

array([ 5, 10, 15, 20,  6, 12, 18, 24,  7, 14, 21, 28,  8, 16, 24, 32])

In [32]:
y = []
for i in x:
    y.append(i-0.1)

array([[  4.9,   5.1],
       [  9.9,  10.1],
       [ 14.9,  15.1],
       [ 19.9,  20.1],
       [  5.9,   6.1],
       [ 11.9,  12.1],
       [ 17.9,  18.1],
       [ 23.9,  24.1],
       [  6.9,   7.1],
       [ 13.9,  14.1],
       [ 20.9,  21.1],
       [ 27.9,  28.1],
       [  7.9,   8.1],
       [ 15.9,  16.1],
       [ 23.9,  24.1],
       [ 31.9,  32.1]])

In [47]:
a = [1,2,3,4,5]
b=[]
a.insert(0,'x')
a.insert(2,'y')
a.insert(4,'z')
a

['x', 1, 'y', 2, 'z', 3, 4, 5]